In [0]:
%%sh

PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF

select *  from market.market_creative_cat_rank_fact_v1 limit 1;
select *  from market.market_creative_log_fact_v1 limit 1;
select *  from market.market_creative_adapp_seen_fact_v1 limit 1;
select *  from market.market_creative_adapp_fact_v2 
limit 1;
EOF




In [0]:


# Country & platform self-consistent

from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config, query    

from conf.settings import CITUS_MKT_NAME, CITUS_MKT_ACCESS_ID, \
    CITUS_MKT_SECRET_KEY, CITUS_MKT_HOSTS
    
import datetime    

CITUS_MKT_NAME='aa_mkt_db'
CITUS_MKT_ACCESS_ID= 'citus_mkt_qa'
CITUS_MKT_HOSTS=[('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com',7432)]
CITUS_MKT_SECRET_KEY='rn*Wh%osCl2C'

citus_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)


def gen_sample_WW(table, columns):
    sql = """
    select  {} from market.{}
    where country_code != 'WW'
    limit 3
    """.format(','.join(columns),table)
    result = query(citus_dsn, sql)
    return result
    
def gen_sample_platform(table, columns):
    sql = """
    select  {} from market.{}
    where ad_platform_id != 1
    limit 3
    """.format(','.join(columns),table)
    result = query(citus_dsn, sql)
    return result
    

def build_conditions(columns, value):
    conditions = []
    for item in zip(columns,value):
        if str == type(item[1]) or datetime.date==  type(item[1]):
            conditions.append(item[0] + "= '"+str(item[1])+"'")
        else:
            conditions.append(item[0] + "= "+str(item[1]))
    return conditions
    
    
def check_platform(table,conditions):
    seen_fact_sql = """
        select  count(distinct(ad_platform_id)) from market.{} 
where ad_platform_id = 1 and {}
    """.format(table, ' and '.join(conditions))
    print(seen_fact_sql)
    result = query(citus_dsn,seen_fact_sql)
    print(result[0][0]==1)

def check_WW(table,conditions):
    seen_fact_sql = """
        select  count(distinct(country_code)) from market.{} 
where country_code = 'WW' and {}
    """.format(table, ' and '.join(conditions))
    print(seen_fact_sql)
    result = query(citus_dsn,seen_fact_sql)
    print(result[0][0]==1)
    
    

def all_platform(table,columns):
    samples = gen_sample_platform(table,columns)
    for item in samples:
        conditions = build_conditions(columns,item)
        check_platform(table,conditions)

def all_WW(table, columns):
    samples = gen_sample_WW(table,columns)
    for item in samples:
        conditions = build_conditions(columns,item)
        check_WW(table,conditions)
        
        
def test_rank_country():
    columns= ['device_code' ,'granularity' ,'date' , 'category_id' ,'ad_app_id', 'creative_id']
    table = 'market_creative_cat_rank_fact_v1'
    all_WW(table,columns)

def test_log_fact_country():
    columns= ['device_code' ,'ad_platform_id' ,'date ', 'ad_app_id', 'creative_id']
    table = 'market_creative_log_fact_v1'
    all_WW(table, columns)

def test_log_fact_platform():
    columns= ['device_code' ,'country_code' ,'date ', 'ad_app_id', 'creative_id']
    table = 'market_creative_log_fact_v1'
    all_platform(table, columns)    

def test_seen_fact_country():
    columns= ['device_code' ,'ad_platform_id' ,'ad_app_id', 'creative_id']
    table = 'market_creative_adapp_seen_fact_v1'
    all_WW(table,columns)    
    

def test_seen_fact_platform():
    columns= ['device_code' ,'country_code' , 'ad_app_id', 'creative_id']
    table = 'market_creative_adapp_seen_fact_v1'
    all_platform(table, columns)        

test_seen_fact_platform()


# print("test_rank")
# test_rank()
# print("test_log_fact")
# test_log_fact()
# print("test_seen_fact")
# test_seen_fact()

In [0]:

# Case 3 - seen_fact seen date

from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config, query    

from conf.settings import CITUS_MKT_NAME, CITUS_MKT_ACCESS_ID, \
    CITUS_MKT_SECRET_KEY, CITUS_MKT_HOSTS

# PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF
CITUS_MKT_NAME='aa_mkt_db'
CITUS_MKT_ACCESS_ID= 'citus_mkt_qa'
CITUS_MKT_HOSTS=[('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com',7432)]
CITUS_MKT_SECRET_KEY='rn*Wh%osCl2C'

citus_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)


from datetime import datetime, timedelta
import calendar


sql = """
select device_code,country_code,ad_app_id,creative_id from market.market_creative_cat_rank_fact_v1
limit 30
"""
result = query(citus_dsn, sql)



def check_rank_ad_platform(device_code,country_code, ad_app_id,creative_id):

    seen_fact_sql = """
        select * from market.market_creative_adapp_seen_fact_v1 
where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}' and country_code='{country_code}'
    """.format(
        ad_app_id=ad_app_id,
        creative_id=creative_id,
        device_code=device_code,
        country_code=country_code
        )
    
    result_seen_fact_sql = query(citus_dsn,seen_fact_sql)


    test_rank_sql = """
        select min(date), max(date)  from market.market_creative_cat_rank_fact_v1 
where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}' and country_code='{country_code}'
    """.format(
        ad_app_id=ad_app_id,
        creative_id=creative_id,
        device_code=device_code,
        country_code=country_code
        )
    result_test_rank_sql = query(citus_dsn, test_rank_sql)
    
    
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(seen_fact_sql)
    print(result_seen_fact_sql)
    print(test_rank_sql)
    print(result_test_rank_sql)
    
    # if len(result_test_rank_sql) != len(result_seen_fact_sql):
    #     print(seen_fact_sql)
    #     print(result_seen_fact_sql)
    #     print(test_rank_sql)
    #     print(result_test_rank_sql)
        
        
    # else:
        # print(True)
    print("=====================================")


for item in result:
    check_rank_ad_platform(item[0],item[1],item[2],item[3])




In [0]:
%%sh

PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF


select min(first_seen_date), max(last_seen_date) from market.market_creative_adapp_seen_fact_v1 
where ad_app_id=530621395 and creative_id=90100014630638 and device_code='ios-tablet' and ad_platform_id = '136' and  country_code !='WW';

select * from market.market_creative_adapp_seen_fact_v1 
where ad_app_id=530621395 and creative_id=90100014630638 and device_code='ios-tablet' and ad_platform_id = '136' and  country_code !='WW';



select * from market.market_creative_adapp_seen_fact_v1 
where ad_app_id=530621395 and creative_id=90100014630638 and device_code='ios-tablet' and ad_platform_id = '136' and  country_code ='WW';

EOF





In [0]:

# seen_fact seen date WW correct first_seen/last_seen_date


from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config, query    

from conf.settings import CITUS_MKT_NAME, CITUS_MKT_ACCESS_ID, \
    CITUS_MKT_SECRET_KEY, CITUS_MKT_HOSTS

CITUS_MKT_NAME='aa_mkt_db'
CITUS_MKT_ACCESS_ID= 'citus_mkt_qa'
CITUS_MKT_HOSTS=[('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com',7432)]
CITUS_MKT_SECRET_KEY='rn*Wh%osCl2C'

citus_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)


from datetime import datetime, timedelta
import calendar


sql = """
select device_code, ad_platform_id,creative_id,ad_app_id,first_seen_date,last_seen_date  from market.market_creative_adapp_seen_fact_v1
where country_code = 'WW'
limit 30
"""
result = query(citus_dsn, sql)



def check_rank_ad_platform(device_code,  ad_platform_id,  creative_id, ad_app_id, first_seen_date, last_seen_date):

    seen_fact_sql = """
        select min(first_seen_date), max(last_seen_date) from market.market_creative_adapp_seen_fact_v1 
where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}' and ad_platform_id = '{ad_platform_id}' and  country_code !='WW'
    """.format(
        ad_app_id=ad_app_id,
        creative_id=creative_id,
        device_code=device_code,
        ad_platform_id=ad_platform_id
        )
    
    result_seen_fact_sql = query(citus_dsn,seen_fact_sql)
    if result_seen_fact_sql[0][0] == first_seen_date and result_seen_fact_sql[0][1] == last_seen_date:
        # print(True)
        pass
    else:
        print (device_code,  ad_platform_id,  creative_id, ad_app_id, first_seen_date, last_seen_date)
        print(seen_fact_sql)
        print(result_seen_fact_sql)
    
    # print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
  


for item in result:
    check_rank_ad_platform(item[0],item[1],item[2],item[3], item[4], item[5])




In [0]:

# Case 2 - Rank consistent with seen_fact

from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config, query    

from conf.settings import CITUS_MKT_NAME, CITUS_MKT_ACCESS_ID, \
    CITUS_MKT_SECRET_KEY, CITUS_MKT_HOSTS

# PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF
CITUS_MKT_NAME='aa_mkt_db'
CITUS_MKT_ACCESS_ID= 'citus_mkt_qa'
CITUS_MKT_HOSTS=[('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com',7432)]
CITUS_MKT_SECRET_KEY='rn*Wh%osCl2C'

citus_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)


from datetime import datetime, timedelta
import calendar



sql = """
select device_code,ad_app_id,creative_id from market.market_creative_adapp_fact_v2
where country_count > 5
limit 3000
"""
result = query(citus_dsn, sql)
# print(result)


def check_rank_ad_platform(device_code,ad_app_id,creative_id):
    test_rank_sql = """
        select distinct country_code  from market.market_creative_cat_rank_fact_v1 
where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}'
    """.format(
        ad_app_id=ad_app_id,
        creative_id=creative_id,
        device_code=device_code
        )
    result_test_rank_sql = query(citus_dsn, test_rank_sql)
    
    log_seen_fact_sql = """
        select distinct country_code  from market.market_creative_adapp_seen_fact_v1 
where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}'
    """.format(
        ad_app_id=ad_app_id,
        creative_id=creative_id,
        device_code=device_code,
        )
    result_log_seen_fact_sql = query(citus_dsn, log_seen_fact_sql)    
    print(">>>>>>>>>>>>>>>>>>>>>")
    if  len(result_log_seen_fact_sql) != len(result_test_rank_sql):

        print(test_rank_sql)
        print(result_test_rank_sql)
        print(log_seen_fact_sql)
        print(result_log_seen_fact_sql)
    
    else:
        print(True)
    print("=======================")


for item in result:
    check_rank_ad_platform(item[0],item[1],item[2])




In [0]:


# Case 1 - Rank consistent with log_fact

from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config, query    

from conf.settings import CITUS_MKT_NAME, CITUS_MKT_ACCESS_ID, \
    CITUS_MKT_SECRET_KEY, CITUS_MKT_HOSTS

# PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF
CITUS_MKT_NAME='aa_mkt_db'
CITUS_MKT_ACCESS_ID= 'citus_mkt_qa'
CITUS_MKT_HOSTS=[('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com',7432)]
CITUS_MKT_SECRET_KEY='rn*Wh%osCl2C'

citus_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)


from datetime import datetime, timedelta
import calendar

def get_start_end_day(input_date, granularity):
    if granularity== 'weekly':
        start = input_date - timedelta(days=input_date.weekday() +1) 
        end = start + timedelta(days=6)
    elif granularity== 'monthly':
        start = input_date.replace(day = 1)
        end = input_date.replace(day = calendar.monthrange(input_date.year, input_date.month)[1])
    return start,end



sql = """
select distinct  device_code,granularity,date,ad_app_id,creative_id from market.market_creative_adapp_fact_v2
where country_count > 5 and date < '2020-01-01'
limit 1000
"""
result = query(citus_dsn, sql)
# print(result)

county_list_result = []
def check_rank_ad_platform(device_code,granularity,date,ad_app_id,creative_id):
    start_day, end_day = get_start_end_day(date,granularity)
    test_rank_sql = """
        select distinct country_code  from market.market_creative_cat_rank_fact_v1 
where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}' and date='{date}' and granularity='{granularity}'
    """.format(
        ad_app_id=ad_app_id,
        creative_id=creative_id,
        device_code=device_code,
        date=date,
        granularity=granularity
        )
    result_test_rank_sql = query(citus_dsn, test_rank_sql)
    
    log_fact_sql = """
        select distinct country_code  from market.market_creative_log_fact_v1 
where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}' and  date>='{start_day}' and date<='{end_day}'
    """.format(
        ad_app_id=ad_app_id,
        creative_id=creative_id,
        device_code=device_code,
        start_day=start_day,
        end_day=end_day
        )
    result_log_fact_sql = query(citus_dsn, log_fact_sql)    
    # print(">>>>>>>>>>>>>>>>>>>>>")
    if  len(result_log_fact_sql) != len(result_test_rank_sql):
        print(device_code,granularity,date,ad_app_id,creative_id)
        print(test_rank_sql)
        print(result_test_rank_sql)
        print(log_fact_sql)
        print(result_log_fact_sql)
        county_list_result.append([[i[0] for i in result_test_rank_sql],[i[0] for i in result_log_fact_sql]])
        return False
    else:
        return True
    
    # print("=======================")、


for item in result:
    is_failed =  check_rank_ad_platform(item[0],item[1],item[2],item[3],item[4])


print(county_list_result)
rank_has_more = 0
log_has_more = 0
rank_log_inconsistance = 0
for item in county_list_result:
    if len(item[0]) - len(item[1]) == len(set(item[0])-set(item[1])):
        rank_has_more = rank_has_more +1
    elif len(item[1]) - len(item[0]) == len(set(item[1])-set(item[0])):
        log_has_more = log_has_more +1
    else:
        rank_log_inconsistance +1

print(len(county_list_result))
print(rank_has_more,log_has_more,rank_log_inconsistance)
    




In [0]:

# Case 5 - rank.county consistence with adapp_fact county
# Case 6 - log_fact.pub_app_id consistence with adapp_fact pub_app_count
# Case 7 - log_fact.ad_platform_id consistence with adapp_fact ad_platform_count

   
from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config, query    

from conf.settings import CITUS_MKT_NAME, CITUS_MKT_ACCESS_ID, \
    CITUS_MKT_SECRET_KEY, CITUS_MKT_HOSTS

# PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF
CITUS_MKT_NAME='aa_mkt_db'
CITUS_MKT_ACCESS_ID= 'citus_mkt_qa'
CITUS_MKT_HOSTS=[('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com',7432)]
CITUS_MKT_SECRET_KEY='rn*Wh%osCl2C'

citus_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)

from datetime import datetime, timedelta
import calendar

def get_start_end_day(input_date, granularity):
    if granularity== 'weekly':
        start = input_date - timedelta(days=input_date.weekday() +1) 
        end = start + timedelta(days=6)
    elif granularity== 'monthly':
        start = input_date.replace(day = 1)
        end = input_date.replace(day = calendar.monthrange(input_date.year, input_date.month)[1])
    return start,end



###############################################
#  Case 1 Country -Start
###############################################
sql = """
select device_code,granularity,date,ad_app_id,creative_id, country_count,country_list,country_code from market.market_creative_adapp_fact_v2 
where country_count > 3
limit 3
"""
result = query(citus_dsn, sql)
# print(result)

def check_rank_country(device_code,granularity,date,ad_app_id,creative_id,country_count,country_list):
    start_day, end_day = get_start_end_day(date,granularity)
    test_sql = """
        select distinct country_code  from market.market_creative_cat_rank_fact_v1 
where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}' and date>='{start_day}' and date<='{end_day}'
    """.format(
        ad_app_id=ad_app_id,
        creative_id=creative_id,
        device_code=device_code,
        start_day =start_day,
        end_day= end_day
        )
    result = query(citus_dsn, test_sql)
    if len(result) == country_count:
        pass
    else:
        print(device_code,granularity,date,ad_app_id,creative_id,country_count,country_list)
        print(test_sql)
        print(result)
    print("\n\n>>>>>>")
        
    

for item in result:
    check_rank_country(item[0],item[1],item[2],item[3],item[4],item[5],item[6])   

###############################################
#  Case 1 Country - End
###############################################


# ###############################################
# #  Case 2 pub_app -Start
# ###############################################
# sql = """
# select device_code,granularity,date,ad_app_id,creative_id, pub_app_count,pub_app_list from market.market_creative_adapp_fact_v2 
# where pub_app_count > 3
# limit 3000
# """
# result = query(citus_dsn, sql)
# # print(result)

# def check_rank_pub_app(device_code,granularity,date,ad_app_id,creative_id,pub_app_count,pub_app_list):
#     start_day, end_day = get_start_end_day(date,granularity)
#     test_sql = """
#         select distinct pub_app_id  from market.market_creative_log_fact_v1 
# where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}' and date>='{start_day}' and date<='{end_day}'
#     """.format(
#         ad_app_id=ad_app_id,
#         creative_id=creative_id,
#         device_code=device_code,
#         start_day =start_day,
#         end_day= end_day
#         )
#     result = query(citus_dsn, test_sql)
#     if len(result) == pub_app_count:
#         # print(True)
#         pass
#     else:
#         print(device_code,granularity,date,ad_app_id,creative_id,pub_app_count,pub_app_list)
#         print(test_sql)
#         print(result)

    

# for item in result:
#     check_rank_pub_app(item[0],item[1],item[2],item[3],item[4],item[5],item[6])

# ###############################################
# #  Case 2 pub_app - End
# ###############################################


# ###############################################
# #  Case 3 ad_platform -Start
# ###############################################

# sql = """
# select device_code,granularity,date,ad_app_id,creative_id, ad_platform_count,ad_platform_list from market.market_creative_adapp_fact_v2 
# where ad_platform_count > 3
# limit 300
# """
# result = query(citus_dsn, sql)
# # print(result)

# def check_rank_ad_platform(device_code,granularity,date,ad_app_id,creative_id,ad_platform_count,ad_platform_list):
#     start_day, end_day = get_start_end_day(date,granularity)
#     test_sql = """
#         select distinct ad_platform_id  from market.market_creative_log_fact_v1 
# where ad_app_id={ad_app_id} and creative_id={creative_id} and device_code='{device_code}' and date>='{start_day}' and date<='{end_day}' and ad_platform_id!=1
#     """.format(
#         ad_app_id=ad_app_id,
#         creative_id=creative_id,
#         device_code=device_code,
#         start_day=start_day,
#         end_day=end_day
#         )
#     result = query(citus_dsn, test_sql)    
#     if len(result) == ad_platform_count:
#         # print(True)
#         pass
#     else:
#         print(device_code,granularity,date,ad_app_id,creative_id,ad_platform_count,ad_platform_list)
#         print(test_sql)
#         print(result)
    
    

# for item in result:
#     check_rank_ad_platform(item[0],item[1],item[2],item[3],item[4],item[5],item[6])

# ###############################################
# #  Case 3 ad_platform - End
# ###############################################    


In [0]:

%sh
#market_creative_adapp_fact_v2 self-consistent

PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF


select device_code,country_code,granularity,date,ad_platform_id,ad_app_id,creative_id,ad_platform_count,ad_platform_list  from market.market_creative_adapp_fact_v2
where
    (ad_platform_count  = 0 AND ad_platform_list is not null)
    OR (ad_platform_count  != 0 AND ad_platform_list is  null)
    OR (ad_platform_count  is null  AND ad_platform_list is not null)
    OR array_length(string_to_array(ad_platform_list, ','),1) <> ad_platform_count
    limit 10;    
    
select device_code,country_code,granularity,date,ad_platform_id,ad_app_id,creative_id,ad_platform_count,ad_platform_list  from market.market_creative_adapp_fact_v2
where ad_platform_count =0 or ad_platform_count is null
limit 10;


select device_code,country_code,granularity,date,ad_platform_id,ad_app_id,creative_id,pub_app_count,pub_app_list  from market.market_creative_adapp_fact_v2
where
    (pub_app_count  = 0 AND pub_app_list is not null)
    OR (pub_app_count  != 0 AND pub_app_list is  null)
    OR (pub_app_count  is null AND pub_app_list is not null)
    OR (array_length(string_to_array(pub_app_list, ','),1) <> pub_app_count and pub_app_count<20)
    limit 10; 

select device_code,country_code,granularity,date,ad_platform_id,ad_app_id,creative_id,pub_app_count,pub_app_list  from market.market_creative_adapp_fact_v2
where  pub_app_count = 0 or pub_app_count is null
limit 10;


select device_code,country_code,granularity,date,ad_platform_id,ad_app_id,creative_id,pub_app_count,pub_app_count,country_count,country_list  from market.market_creative_adapp_fact_v2
where
     (country_count  = 0 AND country_list is not null)
    OR (country_count  != 0 AND country_list is  null)
    OR (country_count  is null AND country_list is not null)
    OR array_length(string_to_array(country_list, ','),1) <> country_count
    limit 10;  

select device_code,country_code,granularity,date,ad_platform_id,ad_app_id,creative_id,pub_app_count,pub_app_count,country_count,country_list  from market.market_creative_adapp_fact_v2
where country_count=0 or country_count is null
limit 10;


EOF